# 의사록 hwp 파일 크롤링 (final)

In [3]:
# 2005년 4월 ~ 2012년 12월 의사록(.hwp) 다운로드 링크 가져오기

from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

def get_hwp_data(startyear, startmonth, endyear, endmonth):
    driver = webdriver.Chrome()
    
    result = []

    try:
        while startyear <= endyear:
            url = f'https://www.bok.or.kr/portal/singl/crncyPolicyDrcMtg/listYear.do?mtgSe=A&menuNo=200755&pYear={startyear}#content'
            driver.get(url)
            time.sleep(3)
            
            tr_tags = driver.find_elements(By.CSS_SELECTOR, '#tableId > tbody > tr')
            
            for tr_tag in tr_tags:
                day = tr_tag.find_element(By.CSS_SELECTOR,'th').text
                month = int(day[:2])

                if startyear == 2005 and month < startmonth:
                    continue
                if endyear == 2012 and month > endmonth:
                    break

                td_tag = tr_tag.find_elements(By.CSS_SELECTOR,'td')[2]
                if startyear == 2006 or (startyear == 2007)&(month == 1):
                    li_tag = td_tag.find_elements(By.CSS_SELECTOR,'div.fileGroupSet>div.fileGoupBox>ul>li')[1]
                    a_tag = li_tag.find_element(By.CSS_SELECTOR,'a')
                    dlink = a_tag.get_attribute('href')
                else:
                    a_tag = td_tag.find_element(By.CSS_SELECTOR,'div.fileGroupSet>div.fileGoupBox>ul>li>a')
                    dlink = a_tag.get_attribute('href')

                result.append(dict(
                    year = startyear,
                    day = day,
                    dlink = dlink
                ))

            startyear += 1
            
    finally:
        driver.quit()

    return result

data = get_hwp_data(2005, 4, 2012, 12)

df = pd.DataFrame(data,columns=['year','day','dlink'])

df

,year,day,dlink
0,2005,04월 07일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...
1,2005,05월 12일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...
2,2005,06월 09일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...
3,2005,07월 07일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...
4,2005,08월 11일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...
...,...,...,...
89,2012,08월 09일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...
90,2012,09월 13일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...
91,2012,10월 11일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...
92,2012,11월 09일(금),https://www.bok.or.kr/portal/cmmn/file/fileDow...


In [4]:
#다운로드 파일에 저장하기

from selenium import webdriver
from selenium.webdriver.common.by import By


driver = webdriver.Chrome()

for i in df['dlink']:
    driver.get(i)

time.sleep(20)

driver.quit()

In [5]:
#filename 리스트 만들기

import os

path = "C:/Users/kwkwo/Downloads/"
all_filenames = os.listdir(path)
hwp_files = []

for filename in all_filenames:
    if filename[-4:] == ".hwp":
        hwp_files.append(filename)

print(hwp_files)

['2005년+제10차+금통위+의사록.hwp', '2005년+제12차+금통위+의사록.hwp', '2005년+제14차+금통위+의사록.hwp', '2005년+제17차+금통위+의사록.hwp', '2005년+제19차+금통위+의사록.hwp', '2005년+제21차+금통위+의사록.hwp', '2005년+제24차+금통위+의사록.hwp', '2005년+제26차+금통위+의사록.hwp', 'munete20.hwp', '금융통화위원회+의사록(2006년도+제10차).hwp', '금융통화위원회+의사록(2006년도+제12차).hwp', '금융통화위원회+의사록(2006년도+제14차).hwp', '금융통화위원회+의사록(2006년도+제16차).hwp', '금융통화위원회+의사록(2006년도+제19차).hwp', '금융통화위원회+의사록(2006년도+제21차).hwp', '금융통화위원회+의사록(2006년도+제23차).hwp', '금융통화위원회+의사록(2006년도+제25차).hwp', '금융통화위원회+의사록(2006년도+제2차).hwp', '금융통화위원회+의사록(2006년도+제4차).hwp', '금융통화위원회+의사록(2006년도+제6차).hwp', '금융통화위원회+의사록(2006년도+제8차).hwp', '금융통화위원회+의사록(2007년도+제11차).hwp', '금융통화위원회+의사록(2007년도+제13차).hwp', '금융통화위원회+의사록(2007년도+제15차).HWP.hwp', '금융통화위원회+의사록(2007년도+제17차).hwp', '금융통화위원회+의사록(2007년도+제19차).hwp', '금융통화위원회+의사록(2007년도+제21차).hwp', '금융통화위원회+의사록(2007년도+제23차).hwp', '금융통화위원회+의사록(2007년도+제25차).hwp', '금융통화위원회+의사록(2007년도+제2차).hwp', '금융통화위원회+의사록(2007년도+제4차).HWP.hwp', '금융통화위원회+의사록(2007년도+제7차).hwp', '금융통화위원회+의사록(2007년도+제9차).HWP.hwp', '금융

In [7]:
import olefile
import zlib
import struct
import os
import re

# 다운로드 폴더 경로 (Windows에서는 기본적으로 사용자 폴더에 있음)
download_folder = os.path.join(os.path.expanduser("~"), "Downloads")

# .hwp 파일 목록 가져오기
hwp_files = [os.path.join(download_folder, f) for f in os.listdir(download_folder) if f.endswith('.hwp')]

# '통화정책방향〉' 이후의 내용만 가져오는 함수
def extract_text_after_keyword(text):
    keyword = "통화정책방향〉"
    if keyword in text:
        # '통화정책방향〉' 이후의 텍스트만 반환
        return text.split(keyword, 1)[1].strip()
    else:
        # 키워드가 없다면 빈 문자열 반환
        return ""

def get_hwp_text(filename):
    f = olefile.OleFileIO(filename)
    dirs = f.listdir()
    # print(dirs)

    # HWP 파일 검증
    if ["FileHeader"] not in dirs or \
            ["\x05HwpSummaryInformation"] not in dirs:
        raise Exception("Not Valid HWP.")

    # 문서 포맷 압축 여부 확인
    header = f.openstream("FileHeader")
    header_data = header.read()
    is_compressed = (header_data[36] & 1) == 1

    # Body Sections 불러오기
    nums = []
    for d in dirs:
        if d[0] == "BodyText":
            nums.append(int(d[1][len("Section"):]))
    sections = ["BodyText/Section" + str(x) for x in sorted(nums)]

    # 전체 text 추출
    text = ""
    for section in sections:
        bodytext = f.openstream(section)
        data = bodytext.read()
        if is_compressed:
            unpacked_data = zlib.decompress(data, -15)
        else:
            unpacked_data = data

        # 각 Section 내 text 추출
        section_text = ""
        i = 0
        size = len(unpacked_data)
        while i < size:
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff
            rec_len = (header >> 20) & 0xfff

            if rec_type in [67]:
                rec_data = unpacked_data[i + 4:i + 4 + rec_len]
                section_text += rec_data.decode('utf-16')
                section_text += "\n"

            i += 4 + rec_len

        text += section_text
        text += "\n"

    return text

contents = []

for hwp in hwp_files:
    # print(hwp)
    text = get_hwp_text(hwp)
    contents.append(text)


# 텍스트 정제
extracted_contents = [extract_text_after_keyword(text) for text in contents]


In [51]:
df['contents']=extracted_contents

df

,year,day,dlink,contents
0,2005,04월 07일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...,(１) 전일 개최된 동향보고회의에서 조사국장이 보고 제54호 - ｢국내외 경제동향｣...
1,2005,05월 12일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...,(１) 전일 개최된 동향보고회의에서 조사국장이 보고 제68호 - ｢국내외 경제동향｣...
2,2005,06월 09일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...,(１) 전일 개최된 동향보고회의에서 조사국장이 보고 제80호 - ｢국내외 경제동향｣...
3,2005,07월 07일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...,(１) 전일 개최된 동향보고회의에서 조사국장이 보고 제94호 - ｢국내외 경제동향｣...
4,2005,08월 11일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...,(１) 전일 개최된 동향보고회의에서 조사국장이 보고 제107호 - ｢국내외 경제동향...
...,...,...,...,...
89,2012,08월 09일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...,(１) 전일 개최된 동향보고회의에서 조사국장이 보고 제31호 - 「국내외 경제동향」...
90,2012,09월 13일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...,(１) 전일 개최된 동향보고회의에서 조사국장이 보고 제42호 - ｢국내외 경제동향｣...
91,2012,10월 11일(목),https://www.bok.or.kr/portal/cmmn/file/fileDow...,(１) 전일 개최된 동향보고회의에서 조사국장이 보고 제32호 - ｢국내외 경제동향｣...
92,2012,11월 09일(금),https://www.bok.or.kr/portal/cmmn/file/fileDow...,( 1 ) 전일 개최된 동향보고회의에서 조사국장이 보고 제47호 - ｢국내외 경제동...
